In [26]:
import pandas as pd
import numpy as np
import statsmodels.stats.multicomp as multi
import statsmodels.formula.api as smf

In [8]:
# nesarc = 'https://d18ky98rnyall9.cloudfront.net/_cf16dab6c94262cc58a6bd4e0f753e56_nesarc_pds.csv?Expires=1581984000&Signature=Ur-iAfRV4~9PdDRQW15BB7nMwWNZJT3eLKgaR8Xg56Y6ysNuHpexkKSGqY45t0NWgGA-GntegmfT3sihNbaGzRnZukYNMNd4cR2DfgGnnIfzxteor5K0Sokowae9pkdkbc~X2IDg0JFp-8nehOuS-89iMcAMVDWhBISDZEq5IXE_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
# nesarc_data =pd.read_csv(nesarc)
nesarc_data.to_csv('data/nesarcdata.csv', index=False)

In [6]:
nesarc_data.shape

(43093, 3008)

In [9]:
addhealth_pds = 'https://d18ky98rnyall9.cloudfront.net/_e9d2d409bf11bb736eb1bd355871c3f6_addhealth_pds.csv?Expires=1581984000&Signature=IFDwyERCk10vX5YmE8HgyaEsUmJR2PCF939KdjZIeaqW4dCd2duQYtcJJFaQoQuGPIuyAob9BO6mwbjakbzoJPd-Sev9G0HNAChSLrKGRW~NNjIKICcrKQjmcXeGQfrsZGaN2S6Fz~sgYrgCgG3T9LD~IkggP9y1VEGhroGLy3U_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A'
addhealth_pds_data = pd.read_csv(addhealth_pds)
addhealth_pds_data.to_csv('data/addhealthpds.csv', index=False)

/Users/royalshoulders/.virtualenvs/PYTHON/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (8,11,22,65,134,135,177,205,206,207,208,346,366,523,532,533,753,755,756,842,843,847,848,849,856,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,966,967,974,979,980,982,986,987,989,991,992,994,1010,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1073,1075,1078,1079,1080,1081,1084,1085,1086,1087,1088,1091,1092,1093,1094,1095,1097,1098,1099,1119,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1184,1185,1186,1187,1190,1191,1192,1193,1194,1197,1198,1199,1200,1201,1219,1221,1222,1235,1236,1237,1238,1239,1240,1262,1263,1264,1265,1266,1267,1268,1269,1270,1

In [22]:
nesarc_data['S3AQ3B1'] = pd.to_numeric(nesarc_data['S3AQ3B1'], errors='coerce')
nesarc_data['S3AQ3C1'] = pd.to_numeric(nesarc_data['S3AQ3C1'], errors='coerce')
nesarc_data['CHECK321'] = pd.to_numeric(nesarc_data['CHECK321'], errors='coerce')

In [32]:
# subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1 = nesarc_data[(nesarc_data['AGE'] >= 18) & 
                  (nesarc_data['AGE'] <= 25) &
                  (nesarc_data['CHECK321'] == 1)].copy()

In [33]:
# setting missing data
sub1['S3AQ3B1'] = sub1['S3AQ3B1'].replace(9, np.nan)
# setting missing data
sub1['S3AQ3C1'] = sub1['S3AQ3C1'].replace(99, np.nan)

In [34]:
# recoding number of days smoked in the past months
recode1 = {1: 30, 2:22, 3: 14, 4: 5, 5:2.5, 6:1}
sub1['USFREQMO'] = sub1['S3AQ3B1'].map(recode1)
sub1['USFREQMO'] = pd.to_numeric(sub1['USFREQMO'], errors='coerce')

In [38]:
# create a secondary variable multiplying the days smoked/month and the number of cig/per day = total cig per month
sub1['NUMCIGMO_EST'] = sub1['USFREQMO'] * sub1['S3AQ3C1']
sub1['NUMCIGMO_EST'] = pd.to_numeric(sub1['NUMCIGMO_EST'], errors='coerce')

# ANOVA 2 LEVEL

In [40]:
model1 = smf.ols(formula='NUMCIGMO_EST ~ C(MAJORDEPLIFE)', data=sub1)
results1 = model1.fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.550
Date:                Sun, 16 Feb 2020   Prob (F-statistic):             0.0597
Time:                        01:28:08   Log-Likelihood:                -11934.
No. Observations:                1697   AIC:                         2.387e+04
Df Residuals:                    1695   BIC:                         2.388e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              312.8380 

In [43]:
sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna().copy()
print('means for numcigmo_est by major depression status')
means_numcigmo = sub2.groupby(['MAJORDEPLIFE']).mean()
print(means_numcigmo)

means for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               312.837989
1               341.375000


In [44]:
print('standard devs for numcigmo_est by major depression status')
std_numcigmo = sub2.groupby(['MAJORDEPLIFE']).std()
print(std_numcigmo)

standard devs for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               269.002344
1               288.495118


So, here I see that young adult smokers without major depression, as indicated by a value of zero, smoke an average of 312.8 cigarettes per month. And that those with major depression, indicated by a value of one, smoke on average, 341.4 cigarettes per month. Because the p-value is greater than 0.05, actually near 0.06, we can choose to accept the null hypothesis and say that these means are statistically equal. And that there is no association between the presence or absence of major depression and the number of cigarettes smoked per month, among young adult smokers. If I had chosen to reject the null hypothesis, I would be wrong nearly six out of a hundred times. And again, by normal scientific standards, this is not adequate certainty to reject the null hypothesis, and say that there is an association. Instead, we're going to accept the null hypothesis and say that there is no association. Had the p-value been less than .05? I would be more confident that there was a significant association.


To interpret a significant association, I would look at the means table. If p would have been less than .05, I can see that the individuals with major depression smoke more than individuals without. And again, with a significant p-value I could have said that young adults smokers with major depression smoke signficantly more cigarettes per month than young adult smokers without major depression.

you will also see a warning message regarding standard errors below the OLS table. Indicating that you can assume that the covariance matrix of the errors is correctly specified.This tells us that standard error estimates are valid. As long as the underlying assumptions about the errors in the OLS regression are met

# ANOVA More than 2 levels

In [46]:
sub3 = sub1[['NUMCIGMO_EST', 'ETHRACE2A']].dropna().copy()

model2 = smf.ols(formula= 'NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub3)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     24.40
Date:                Sun, 16 Feb 2020   Prob (F-statistic):           1.18e-19
Time:                        01:46:49   Log-Likelihood:                -11888.
No. Observations:                1697   AIC:                         2.379e+04
Df Residuals:                    1692   BIC:                         2.381e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           368.7865      8.22

In [48]:
means_numcigmo2 = sub3.groupby(['ETHRACE2A']).mean()
means_numcigmo2

,NUMCIGMO_EST
ETHRACE2A,
1,368.786528
2,259.273810
3,310.988095
4,244.258621
5,219.758258


The F-test and the p-value do not provide insight into why the null hypothesis can be rejected because there are multiple levels to my categorical explanatory variable. They do not tell us in what way the population means are not statistically equal. 


# Tukey HSD for more than 2 level

In [49]:
import statsmodels.stats.multicomp as multi

In [50]:
mc1 = multi.MultiComparison(sub3['NUMCIGMO_EST'], sub3['ETHRACE2A'])
result3 = mc1.tukeyhsd()
print(result3.summary())

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower     upper   reject
---------------------------------------------------------
     1      2 -109.5127  0.001 -164.6441  -54.3814   True
     1      3  -57.7984 0.6251 -172.5914   56.9945  False
     1      4 -124.5279 0.0051 -222.9229  -26.1329   True
     1      5 -149.0283  0.001   -194.89 -103.1665   True
     2      3   51.7143 0.7555  -71.6021  175.0307  False
     2      4  -15.0152    0.9  -123.233   93.2026  False
     2      5  -39.5156 0.4492 -103.8025   24.7714  False
     3      4  -66.7295 0.7058 -214.5437   81.0848  False
     3      5  -91.2298 0.2269 -210.6902   28.2305  False
     4      5  -24.5004    0.9 -128.3027    79.302  False
---------------------------------------------------------


differences in smoking quantity for each ethnic group pair. In the first row of the table, we see the comparison between ethnic group one and two. Individuals endorsing white ethnicity versus those endorsing black ethnicity. As well as mean differences in number of cigarettes smoked between these two groups. Python has calculated a P value, though it is not displayed, that takes the multiple comparisons into consideration and protects us from inflating our type 1 error and rejecting the null hypothesis when the null hypothesis is true.

So we can see that ethnic group one is significantly different than ethnic groups two, four, and five. And when we again examine group means, we can say that individuals endorsing white ethnicity, group one, smoke significantly more cigarettes per month, than individuals endorsing black, asian and hispanic ethnicity. Groups two, four and five.